<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/04_vgg_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Overview: CIFAR10 Classification with a Custom VGG-like Network

This notebook guides you through building a VGG-style convolutional neural network from scratch to classify images in the CIFAR10 dataset. The workflow includes:

- **Importing Libraries**: Key Python libraries for deep learning, data handling, and visualization.
- **Loading Data**: CIFAR10 dataset is loaded, containing 60,000 color images across 10 classes.
- **Preprocessing**: Images are normalized to the [0, 1] range for better model performance.
- **Visualization**: 25 sample images are displayed with their class names for a quick visual check.
- **Model Construction**: A VGG-like CNN is built from scratch using TensorFlow/Keras, with multiple convolutional and pooling layers, followed by dense layers.
- **Training**: The model is trained for 30 epochs on the CIFAR10 data.
- **Evaluation**: Training progress is visualized with accuracy and loss plots. The model is evaluated on the test set to report accuracy.
- **Confusion Matrix**: Predictions are made on the test set and a confusion matrix is plotted to show class-wise performance.

This notebook helps learners understand how to design, train, and evaluate deeper CNN architectures for image classification tasks.

In [ ]:
# Import required libraries
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [ ]:
# Load the CIFAR10 dataset
(train_images, train_labels), (test_images, test_labels) = keras.datasets.cifar10.load_data()

In [ ]:
# Normalize pixel values to [0, 1]
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# Define class names for CIFAR10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
# Display 25 sample images
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.imshow(train_images[i])
    plt.axis('off')
    plt.title(class_names[train_labels[i,0]])
plt.show()

In [ ]:
# Build a VGG-like model from scratch
# This is a simplified version for CIFAR10

def build_vgg_like(input_shape, num_classes):
    model = keras.Sequential()
    # Block 1
    model.add(keras.layers.Conv2D(64, (3,3), activation='relu', padding='same', input_shape=input_shape))
    model.add(keras.layers.Conv2D(64, (3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    # Block 2
    model.add(keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(keras.layers.Conv2D(128, (3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    # Block 3
    model.add(keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(keras.layers.Conv2D(256, (3,3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D((2,2)))
    # Flatten and Dense layers
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(512, activation='relu'))
    model.add(keras.layers.Dropout(0.5))
    model.add(keras.layers.Dense(num_classes, activation='softmax'))
    return model

In [ ]:
# Create the VGG-like model
model = build_vgg_like(train_images.shape[1:], 10)
model.summary()

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
history = model.fit(train_images, train_labels, epochs=30,
                    batch_size=64,
                    validation_data=(test_images, test_labels),
                    verbose=2)

In [ ]:
# Convert training history to DataFrame
history_df = pd.DataFrame(history.history)

In [ ]:
# Plot training and validation accuracy
sns.lineplot(data=history_df[['accuracy','val_accuracy']])
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.show()

In [ ]:
# Plot training and validation loss
sns.lineplot(data=history_df[['loss','val_loss']])
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc:.3f}')

In [ ]:
# Predict and show confusion matrix
from sklearn.metrics import confusion_matrix
import numpy as np
pred_labels = np.argmax(model.predict(test_images), axis=1)
cm = confusion_matrix(test_labels, pred_labels)
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()
